# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [26]:
from pymystem3 import Mystem
import gensim
from judicial_splitter import split_paragraph, get_sentences
import string
import json
import os
from gensim import matutils
import numpy as np 
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec, KeyedVectors
import warnings
warnings.filterwarnings('ignore')
mystem = Mystem()

In [2]:
# если модель без тэгов
model = Word2Vec.load('/Users/Stoneberry/Desktop/Uni/Прога/4 курс/Поиск/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [ ]:
# если модель с POS-тэггингом
model2 = KeyedVectors.load_word2vec_format('/Users/Stoneberry/Desktop/Uni/Прога/4 курс/Поиск/taiga_upos_skipgram_300_2_2018.vec', binary=False)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [3]:
russian_stopwords = set(stopwords.words('russian'))

Делим тексты на параграфы

In [4]:
def splitting(main_dir, stopwords={}, del_stop=True):
    for root, dirs, files in os.walk(main_dir):
        for name in files:
            if not '.DS_Store' in name:
                with open(os.path.join(root, name), 'r', encoding='utf-8') as f:
                    text = f.read() 
                    parts = split_paragraph(get_sentences(text), 4)
                    for part in parts:
                        part_proc = preprocessing(part, stopwords, del_stopwords=del_stop, del_digit=True)
                        yield (part_proc, name, part)

In [5]:
import re

def preprocessing(input_text, stopwords={}, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [6]:
# если нет слова в модели, то возврашается пустой массив

def get_w2v_vectors(text, model):
    """
    Получает вектор документа
    Не работает
    """
    vec = 0
    lenght = 0
    
    for word in text:
        try: 
            vec += model.wv[word]
            lenght += 1
        except: None
    
    if lenght != 0:
        return vec / lenght
    return [0] * 300


def get_w2v_vectors(lemmas, model): 
    """
    Получает вектор документа
    Работает
    """
    lemmas_vectors = []
    for lemma in lemmas:
        try:
            lemmas_vectors.append(model.wv[lemma])
        except:
            None
    if lemmas_vectors:
        doc_vec = sum(lemmas_vectors)
        normalized_vec = matutils.unitvec(doc_vec)
        return list(normalized_vec)
    else: 
        return [0] * 300


def save_w2v_base(main_dir, model, stopwords={}, del_stop=True):
    """Индексирует всю базу для поиска через word2vec"""
    
    all_data = []

    for part in splitting(main_dir, stopwords=stopwords, del_stop=del_stop):
        vec_info = {}
        vec_info['id'] = part[1]
        vec_info['text'] = part[2]
        vec_info['vec'] = get_w2v_vectors(part[0], model)
        all_data.append(vec_info)
    return all_data #json.dumps(all_data)

In [8]:
main_dir = '/Users/Stoneberry/Desktop/Uni/Прога/4 курс/Поиск/лекции/test'

In [9]:
data1 = save_w2v_base(main_dir, model, stopwords=russian_stopwords)

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from math import log

In [19]:
def tagged_data_creator(main_dir, stopwords={}, del_stop=False):

    tagged_data = []
    i = 0
    for part in splitting(main_dir, stopwords=stopwords, del_stop=del_stop):
        tagged_data.append(TaggedDocument(words=part[0], tags=[i]))
        #print('Complited: ' + str(part[2]) + ' '  + part[1])
        i += 1
    return tagged_data   

In [12]:
def train_doc2vec(tagged_data):
    
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, 
                min_alpha=0.025, epochs=100, workers=2, dm=1)
    
    print('building vocabulary')
    model.build_vocab(tagged_data)
    print('starting training...')
    model.random.seed(42)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    print('model is trained')
    return model

In [27]:
import pickle


model_data = tagged_data_creator('/Users/Stoneberry/Desktop/Uni/Прога/4 курс/Поиск/лекции/train', stopwords=russian_stopwords, del_stop=False)
with open('tagged_data.pickle', 'wb') as f:
    pickle.dump(model_data , f)

my_model = train_doc2vec(model_data)

building vocabulary
starting training...
model is trained


### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [28]:
def get_d2v_vectors(model, text):
    """Получает вектор документа"""
    return model.infer_vector(text)
    

def save_d2v_base(main_dir, model, stopwords={}, del_stop=True):
    """Индексирует всю базу для поиска через doc2vec"""
    
    all_data = []

    for part in splitting(main_dir, stopwords=stopwords, del_stop=del_stop):
        vec_info = {}
        vec_info['id'] = part[1]
        vec_info['text'] = part[2]
        vec_info['vec'] = get_d2v_vectors(model, part[0])
        all_data.append(vec_info)
    return all_data #json.dumps(all_data)

In [29]:
data2 = save_d2v_base(main_dir, my_model, stopwords=russian_stopwords, del_stop=False)

# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [30]:
def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)


def similarity(vec1, vec2):
    return np.dot(vec1, vec2)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [31]:
from collections import defaultdict, Counter

In [32]:
def culc_sim_score(data1, vec):
    
    """
    соединяю параграфы в текст
    """
    
    answer = defaultdict(list) # id : [text, sim_score]
    
    for part in data1:
        obj = answer[part['id']]
        sim = similarity(part['vec'], vec)
        if obj == []:
            obj.append('')
            obj.append(float('-inf'))
        obj[0] += part['text'] + ' '
        if sim > obj[1]:
            obj[1] = sim
    return answer

In [33]:
def search_w2v(string, model, data1, stopwords={}, amount=10, del_stop=True):
    
    if not isinstance(string, str):
        raise ValueError('enter correct data')
    
    words = preprocessing(string, stopwords=stopwords, del_stopwords=del_stop, del_digit=True)
    vec = get_w2v_vectors(words, model)
    answer = culc_sim_score(data1, vec)
    
    for index, ans in enumerate(sorted(answer.items(), key=lambda x: x[1][1], reverse=True)):
        if index >= amount: break
        yield (ans[0], ans[1][0], ans[1][1])


def search_d2v(string, model, data2, stopwords={}, del_stop=False, amount=10):
    
    if not isinstance(string, str):
        raise ValueError('enter correct data')
    
    words = preprocessing(string, stopwords=stopwords, del_stopwords=del_stop, del_digit=True)
    vec = get_d2v_vectors(model, words)
    answer = culc_sim_score(data2, vec)
    
    for index, ans in enumerate(sorted(answer.items(), key=lambda x: x[1][1], reverse=True)):
        if index >= amount: break
        yield (ans[0], ans[1][0], ans[1][1])

In [50]:
for i in search_w2v('уговор', model, data1, del_stop=True, stopwords=russian_stopwords, amount=5):
    print(i[0], i[2])

99998.txt 0.3184982
4.txt 0.31830204
2.txt 0.31830204
3.txt 0.3129466
6.txt 0.3129466


In [49]:
for i in search_d2v('уговор', my_model, data2, del_stop=False, stopwords=russian_stopwords, amount=5):
    print(i[0], i[2])

99999.txt 0.1100643
99998.txt 0.105139665
99879.txt 0.098483324
1.txt 0.08321444
5.txt 0.056647025


После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

##### Okapi

In [92]:
def counter(main_dir, del_stop=True, stopwords={}):

    word_count = defaultdict(dict) # word : {id, count}
    id_text = defaultdict(list) # id : [len, text]

    for root, dirs, files in os.walk(main_dir):
        for name in files:
            if not '.DS_Store' in name:
                with open(os.path.join(root, name), 'r', encoding='utf-8') as f:
                    text = f.read() 
                    words = preprocessing(text, stopwords=stopwords, del_stopwords=del_stop, del_digit=True)
                    id_text[name] += [len(words), text]
                    prob = Counter(words)
                    for word in prob:
                        word_count[word][name] = prob[word]
                
    return word_count, id_text


def score_BM25(qf, dl, avgdl, k1, b, N, n):
    """
    Compute similarity score between search query and documents from collection
    :return: score

    qf - кол - во вхождений слова в документе
    dl - длина документа

    """

    tf = qf / dl
    idf = log(N - n + 0.5 / n + 0.5)
    a = (k1 + 1) * tf
    b = tf + k1*(1 - b + b*(dl / avgdl))

    return (a / b) * idf


def compute_sim(text, doc, id_text, word_count, N, stopwords={}, del_stop=True):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """

    if not isinstance(text, str):
        raise ValueError('enter correct data')
    if not isinstance(doc, str):
        raise ValueError('enter correct data')

    opr = [' ', '  ', '\t', '\n']
    k1 = 2.0
    b = 0.75
    avgdl = np.mean([i[0] for i in id_text.values()])

    ans = 0

    words = preprocessing(text, stopwords=stopwords, del_stopwords=del_stop, del_digit=True)

    for word in words:
        if word in word_count:
            if doc in word_count[word]:
                qf = word_count[word][doc]
            else:
                qf = 0
            dl = id_text[doc][0]
            n = len(word_count[word])
            ans += score_BM25(qf, dl, avgdl, k1, b, N, n)

    return ans


def get_search_result(text, id_text, word_count, stopwords={}, del_stop=True, amount=10):
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """

    if not isinstance(text, str):
        raise ValueError

    arr = []
    N = len(id_text)
   
    for doc in id_text:
        arr.append((doc, id_text[doc][1], compute_sim(text, doc, id_text, word_count, N, stopwords=stopwords, del_stop=del_stop)))
    
    arr = sorted(arr, key=lambda x: x[2], reverse=True)
    
    return arr[:amount]

In [87]:
word_count, id_text = counter(main_dir, stopwords=russian_stopwords)

In [93]:
for i in get_search_result('уговор', id_text, word_count, stopwords=russian_stopwords, amount=5):
    print(i[0], i[2])

1.txt 0
99997.txt 0
99998.txt 0
4.txt 0
99999.txt 0


Так как этого слова нет в той части текстов, на которой я обучала

In [94]:
for i in get_search_result('суд', id_text, word_count, stopwords=russian_stopwords, amount=5):
    print(i[0], i[2])

1.txt 0.03491414211676592
2.txt 0.02815044770681393
99998.txt 0.02262174018738026
3.txt 0.022539850630834624
99997.txt 0.018944873627858447


### Search

In [77]:
def search(string, search_method, amount=10, del_stop=False, stopwords={}):

    if search_method == 'inverted_index':
        global word_count, id_text
        search_result = get_search_result(string, id_text, word_count, del_stop=del_stop, stopwords=stopwords, amount=amount)

    elif search_method == 'word2vec':
        global model, data1
        #model = Word2Vec.load('/Users/Stoneberry/Desktop/Uni/Прога/4 курс/Поиск/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')
        #data = save_w2v_base(main_dir, model, stopwords=stopwords, del_stop=del_stop)
        search_result = [i for i in search_w2v(string, model, data1, stopwords=stopwords, del_stop=del_stop, amount=amount)]

    elif search_method == 'doc2vec':
        global my_model, data2
        #data = save_d2v_base(main_dir, my_model, stopwords=stopwords, del_stop=del_stop)
        search_result = [i for i in search_d2v(string, my_model, data2, del_stop=del_stop, stopwords=stopwords, amount=amount)]

    else:
        raise TypeError('unsupported search method')
    
    return search_result

In [80]:
for i in search('суд', 'word2vec', amount=5, del_stop=False, stopwords=russian_stopwords):
    print(i[0], i[2])

3.txt 0.6987984
6.txt 0.6987984
4.txt 0.6973086
2.txt 0.6973086
99879.txt 0.64392644


In [79]:
for i in search('суд', 'doc2vec', amount=5, del_stop=False, stopwords=russian_stopwords):
    print(i[0], i[2])

99998.txt 13.215175
99879.txt 11.782065
99999.txt 11.243983
99997.txt 10.303209
2.txt 10.15325


In [95]:
for i in search('суд', 'inverted_index', amount=5, del_stop=False, stopwords=russian_stopwords):
    print(i[0], i[2])

1.txt 0.03491414211676592
2.txt 0.02815044770681393
99998.txt 0.02262174018738026
3.txt 0.022539850630834624
99997.txt 0.018944873627858447
